In [1]:
%matplotlib inline
# Imports
import numpy as np
import matplotlib.pyplot as plt
import os
from itertools import product
from code.Utils import ESFitness

# Default variable definitions
pop_sizes = [6, 8, 12, 16, 24]
dimensionalities = [10, 20]
functions = [3, 9, 14, 17, 23]

available_files = product(pop_sizes, dimensionalities, functions)
# final_results_names = ['time_spent', 'generation_sizes', 'sigma', 'best_result', 'best_fitness']

folder_name = 'C:\\src\\master-thesis\\experiments\\pop_size'
data_file_name = 'raw_data\\{pop_size}\\GA_results_{ndim}dim_f{fid}.tdat'
# stat_file_name = 'raw_data\\{pop_size}\\final_GA_results_{ndim}dim_f{fid}.tdat'
plot_data_file_name = 'processed_data\\plot_data.npz'
plot_file_prefix = 'plots\\'

plot_file_extensions = ['.png', '.pdf']
fig_size = (8,6)

os.chdir(folder_name)

In [2]:
fitnesses = {}
FCEs = {}
ERTs = {}

# Obtain FCE/ERT data from data files through ESFitness objects
for pop_size, ndim, fid in available_files:
    with open(data_file_name.format(pop_size=pop_size, ndim=ndim, fid=fid), 'r') as f:
        lines = [line for line in f]
    objects = [eval(line.split('\t')[-1]) for line in lines]
    FCEs[(pop_size, ndim, fid)] = [obj.FCE for obj in objects]
    ERTs[(pop_size, ndim, fid)] = [obj.ERT for obj in objects]
    fitnesses[(pop_size, ndim, fid)] = [(obj.FCE, obj.ERT) for obj in objects]

np.savez(plot_data_file_name, available_files=available_files, fitnesses=fitnesses, FCEs=FCEs, ERTs=ERTs)

In [3]:
plot_data = np.load(plot_data_file_name)
FCEs = plot_data['FCEs'].item()
ERTs = plot_data['ERTs'].item()

for problem in product(dimensionalities, functions):
    ndim, fid = problem

    plot_file_name = "pop_size_plot_{}dim_f{}".format(ndim, fid)
    plt.figure(figsize=fig_size)
    plt.title("Impact of $\lambda$ in {}dim F{}".format(ndim, fid))
    plt.xlabel("Number of evaluations")
    
    for pop_size in pop_sizes:
        x = range(50)
        orig_x = [i for i in x for _ in range(pop_size)]  # [0, ..., 0, 1, ..., 1, ..., 49, ..., 49]
        
        orig_data = FCEs[(pop_size, ndim, fid)]
        temp_data = [np.min(orig_data[i*pop_size:(i+1)*pop_size]) for i in x]
        data = [np.min(temp_data[:i+1]) for i in x]
        plt.plot(x, data, '-', label='$\lambda$ = {}'.format(pop_size))
        plt.ylabel('FCE')
        plt.yscale('log')
        plt.legend(loc=0)

    plt.tight_layout()
    for extension in plot_file_extensions:
        plt.savefig(plot_file_prefix + plot_file_name + extension)
    plt.close()